<a href="https://colab.research.google.com/github/levongocanh/CS114.K21/blob/master/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sarcasm Detection:
Input: một đoạn văn bản

Output: đoạn văn bản có tính châm biếm hay không

Dataset: dùng dataset có sẵn trên kaggle [here](https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection). Dataset được thu thập từ [TheOnion](https://www.theonion.com/)- tin tức châm biếm và [HuffPost](https://www.huffpost.com/)- tin tức chính thống, không châm biếm.

# Đọc data:

In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import re
#đọc dữ liệu từ 2 bộ dataset được lấy trên kaggle
df1 = pd.read_json("/content/Sarcasm_Headlines_Dataset.json", lines=True)
df2 = pd.read_json("/content/Sarcasm_Headlines_Dataset_v2.json", lines=True)

In [22]:
df1.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [23]:
df2.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [24]:
df = pd.concat([df1, df2]) #gộp 2 dataframe thành 1

In [25]:
df

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0
...,...,...,...
28614,https://www.theonion.com/jews-to-celebrate-ros...,jews to celebrate rosh hashasha or something,1
28615,https://local.theonion.com/internal-affairs-in...,internal affairs investigator disappointed con...,1
28616,https://www.huffingtonpost.com/entry/andrew-ah...,the most beautiful acceptance speech this week...,0
28617,https://www.theonion.com/mars-probe-destroyed-...,mars probe destroyed by orbiting spielberg-gat...,1


# Xử lý data:

In [26]:
df = df.drop(['article_link'], axis=1) #xóa cột article_link
df = df.drop_duplicates(['headline']) #xóa những hàng bị trùng ở cột headline

In [27]:
df

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
28497,tyson holds contest to let fans submit new ide...,1
28509,increasingly cocky bernie sanders announces he...,1
28520,cash-strapped zuckerberg forced to sell 11 mil...,1
28544,grocery store bar actually has great little ha...,1


In [28]:
df['headline'] = df['headline'].apply(lambda x: x.lower()) #chuyển đổi nội dung trong cột headline sang chữ thường
df['headline'] = df['headline'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) #loại bỏ ký tự đặc biệt

In [29]:
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret bl...,0
1,the roseanne revival catches up to our thorny ...,0
2,mom starting to fear sons web series closest t...,1
3,boehner just wants wife to listen not come up ...,1
4,jk rowling wishes snape happy birthday in the ...,0


In [30]:
tf = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,3), max_features= 5000)
X = tf.fit_transform(df['headline'])
Y = df['is_sarcastic']

In [31]:
#chia data để train:test tỉ lệ 7:3
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)

# Chọn model train và kiểm tra performance

RandomForestClassifier

In [32]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier()
model_rfc.fit(x_train, y_train)
y_pred = model_rfc.predict(x_test)
f1_score(y_pred, y_test)

0.7426296803084961

LinearSVC

In [60]:
from sklearn.svm import LinearSVC
model_SVC = LinearSVC()
model_SVC.fit(x_train, y_train)
y_pred = model_SVC.predict(x_test)
f1_score(y_pred, y_test)

0.76140350877193

Linear classifiers

In [34]:
from sklearn.linear_model import SGDClassifier
model_sgd = SGDClassifier()
model_sgd.fit(x_train, y_train)
y_pred = model_sgd.predict(x_test)
f1_score(y_pred, y_test)

0.7605090628615502

Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression
model_lg = LogisticRegression()
model_lg.fit(x_train, y_train)
y_pred = model_lg.predict(x_test)
f1_score(y_pred, y_test)

0.7608779360800924